In [1]:
#!pip install numpy pandas matplotlib pillow wordcloud
#!pip install --upgrade Pillow
#!pip install --upgrade wordcloud

import pandas as pd
import numpy as np
import math

import matplotlib.pyplot as plt
import seaborn as sns

import nltk 
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
import re
from nltk import pos_tag
from nltk.corpus import wordnet

from imblearn.over_sampling import RandomOverSampler
import imblearn.pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer,\
      TfidfTransformer, TfidfVectorizer


from wordcloud import WordCloud

#nltk.download("stopwords")
nltk.download("vader_lexicon")
nltk.download("wordnet")

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    



from nltk.tokenize import word_tokenize 
from nltk import pos_tag
from nltk.corpus import wordnet

import string
import re


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,\
HashingVectorizer
from sklearn.model_selection import train_test_split


AttributeError: module 'numpy.typing' has no attribute 'NDArray'

In [3]:
df = pd.read_csv('twitter_data_utf8.csv')

### Data Understanding and Cleaning

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [4]:
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [5]:
#changing column names since they are lenghty 
df.columns = ["tweet", "products", "emotion"]
df.head()

,tweet,products,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [6]:
#check for missing values in tweet column
df[pd.isna(df["tweet"])]

,tweet,products,emotion
6,NaN,NaN,No emotion toward brand or product


In [7]:
#drop this row
df.dropna(subset=["tweet"], inplace=True)

In [8]:
#check for missing value in row
df[pd.isna(df['products'])]

,tweet,products,emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
16,Holler Gram for iPad on the iTunes App Store -...,NaN,No emotion toward brand or product
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,No emotion toward brand or product
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product
34,Anyone at #SXSW who bought the new iPad want ...,NaN,No emotion toward brand or product
...,...,...,...
9087,"@mention Yup, but I don't have a third app yet...",NaN,No emotion toward brand or product
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [9]:
#checking the percentage of NaN in the product column

missing_products_percentage = (df['products'].isna().sum() / len(df)) * 100
print(round(missing_products_percentage, 2))

63.8


In [10]:
# Replacing the NaNs with "Unknown"
df['products'].fillna('Unknown', inplace=True)

In [11]:
#verifying that null values have been handled
df.isna().sum()

tweet       0
products    0
emotion     0
dtype: int64

In [12]:
df['products'].value_counts()

Unknown                            5801
iPad                                946
Apple                               661
iPad or iPhone App                  470
Google                              430
iPhone                              297
Other Google product or service     293
Android App                          81
Android                              78
Other Apple product or service       35
Name: products, dtype: int64

In [13]:

data = {'iPad':946, 'Apple':661, 'iPad or iPhone App':470, 'Google':430, 'Iphone':297,
        'Other Google product/service':293, 'Android App':81, 'Android':78, 
        'Other Apple product/service':35}
courses = list(data.keys())
values = list(data.values())
 
fig = plt.figure(figsize = (20, 10))

plt.bar(courses, values, color ='maroon', 
        width = 0.3)

plt.xlabel("Product")
plt.ylabel("No. of Tweets")
plt.title("Tweet emotion by Product")
plt.show()

NameError: name 'plt' is not defined

In [14]:
# Similar to the initial column names, we will clean the values in the emotion column 
# for easy interpretability and to shorten the amount of typing for the code.

emotion_dict = {'Positive emotion': 'Positive', 'Negative emotion': 'Negative', 
                'No emotion toward brand or product': 'Neutral', 
                "I can't tell": 'Unknown'}
df['emotion'] = df['emotion'].map(emotion_dict)
df.head()

,tweet,products,emotion
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive


In [15]:
df['emotion'].value_counts()

Neutral     5388
Positive    2978
Negative     570
Unknown      156
Name: emotion, dtype: int64

In [16]:

data = { 'Neutral':5388, 'Positive':2978, 'Negative':570, 'Unknown':156}
courses = list(data.keys())
values = list(data.values())
 
fig = plt.figure(figsize = (10, 5))

# creating the bar plot
plt.bar(courses, values, color ='purple', 
        width = 0.4)

plt.xlabel("Emotion")
plt.ylabel("No. of Tweets")
plt.title("Tweet Emotions")
plt.show()

NameError: name 'plt' is not defined

In [17]:
# 'Unknown' makes up a very small portion of tweets and will not be useful in our analysis so we drop them

df = df[df['emotion']!='Unknown']
df['emotion'].value_counts()

Neutral     5388
Positive    2978
Negative     570
Name: emotion, dtype: int64

## Data Exploration

#### Given the business proble, we will analyze positive vs negative tweets as well as Apple vs Google tweets. We will start with positive tweets

#### Tokenization

In [18]:

df_positive = df[df['emotion']=='Positive']
#verifying that neutral and negative tweets have been removed
df_positive['emotion'].value_counts()

Positive    2978
Name: emotion, dtype: int64

In [19]:
corpus_pos = df_positive['tweet'].to_list()
corpus_pos[:5]

["@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",
 '@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.',
 "@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",
 '#SXSW is just starting, #CTIA is around the corner and #googleio is only a hop skip and a jump from there, good time to be an #android fan',
 'Beautifully smart and simple idea RT @madebymany @thenextweb wrote about our #hollergram iPad app for #sxsw! http://bit.ly/ieaVOB']

In [20]:
# TweetTokenizer has built-in functionality such as processing handles and hashtags.

from nltk import TweetTokenizer
import string

#functionize tokenization of tweets
def tokenize_tweets(corpus, preserve_case=False, strip_handles=True):
    """Function returns tokens based on a corpus passed in. The corpus will be broken 
    down into tokens based on TweetTokenizer from the nltk package.
    -------------------------------
    Arguments:
    corpus: the collection of words to be tokenized in a corpus format.
    preserve_case: whether to keep the upper case letters in the words as upper case
    stip_handles: whether to remove twitter handles"""
    
    tokenizer = TweetTokenizer(preserve_case=preserve_case, 
                               strip_handles=strip_handles)
    tokens = tokenizer.tokenize(','.join(corpus))
    return tokens

In [21]:
#tokenize positive tweets
tokens_pos = tokenize_tweets(corpus_pos)

In [22]:
#displaying 20 most common tokens
from nltk import FreqDist
freq = FreqDist(tokens_pos)
freq.most_common(20)

[(',', 4062),
 ('#sxsw', 2991),
 ('.', 2235),
 ('the', 1597),
 ('!', 1245),
 ('link', 1217),
 ('{', 1213),
 ('}', 1213),
 ('to', 1160),
 ('at', 1022),
 ('ipad', 1013),
 ('rt', 933),
 ('for', 908),
 ('a', 792),
 ('apple', 713),
 ('is', 647),
 ('of', 639),
 ('in', 631),
 ('google', 604),
 ('and', 584)]

#### Lemmatize

In [30]:
def lemmatize_tokens(tokens_list):
  
    lemmatizer = WordNetLemmatizer()
    tokens_lemm = [lemmatizer.lemmatize(word) for word in tokens_list]
    return tokens_lemm

In [31]:
#lemmatizing positive tweet tokens
tokens_pos_lemm = lemmatize_tokens(tokens_pos)

NameError: name 'WordNetLemmatizer' is not defined

In [32]:
freq = FreqDist(tokens_pos_lemm)
freq.most_common(30)

NameError: name 'tokens_pos_lemm' is not defined

#### Punctuation and Stopword removal

In [33]:
stop_lst = stopwords.words('english')
stop_lst += list(string.punctuation)
#removing additional punctuation and stopwords that have no meaning
additional_punc = ['“','”','...',"''",'’','``','']
stop_lst += additional_punc
stop_lst += ['#sxsw', '#sxswi', 'sxsw']

NameError: name 'stopwords' is not defined

In [34]:
# function to remove of stop words
def remove_stop(tokens, stop_list=stop_lst):
    
# encoding/decoding tokens to eliminate unrecognized symbols and 
# eliminating external links
    tokens_stopped = [word.encode('ascii','ignore').decode() 
                      for word in tokens 
                      if (word not in stop_list) & 
                      (word.startswith('http') == False)]
    
    return tokens_stopped

NameError: name 'stop_lst' is not defined

In [35]:
tokens_pos_ls = remove_stop(tokens_pos_lemm)

NameError: name 'remove_stop' is not defined

In [36]:

freq = FreqDist(tokens_pos_ls)
freq.most_common(50)

NameError: name 'tokens_pos_ls' is not defined

In [31]:
### ASDFKJASDKFJASDKFASDKFJASDKFJASKDFJASD

In [32]:
#defining a function to find the n most common tokens

def find_frequent(tokens, n=50):
    
    freq = FreqDist(tokens)
    display(freq.most_common(n))

In [ ]:
find_frequent(tokens_pos_ls)

In [ ]:


#defining a function for wordcloud generation
def generate_wordcloud(tokens, collocations=False, background_color='black', 
                       colormap='Greens', display=True):
    
    """Function generates and returns a wordcloud based on a tokens list passed in.
    -------------------------------
    Arguments:
    tokens: a tokens list
    collocations: Whether to include collocations (bigrams) of two words
    background_color: background color of the resulting word cloud
    colormap: the color map for the words that will be in the word cloud
    display: Whether to show the resulting wordcloud"""
    
    ## Initalize a WordCloud
    wordcloud = WordCloud(collocations=collocations, 
                          background_color=background_color, 
                          colormap=colormap, 
                          width=500, height=300)

    ## Generate wordcloud from tokens
    wordcloud.generate(','.join(tokens))

    ## Plot with matplotlib
    if display:
        plt.figure(figsize = (12, 15), facecolor = None) 
        plt.imshow(wordcloud) 
        plt.axis('off');
    return wordcloud

In [ ]:
cloud_pos_w_company = generate_wordcloud(tokens_pos_ls, collocations=True)

In [ ]:
#create subsets of positive and negative tweets

#positive_df = df[df["emotion"]=="Positive emotion"]
#negative_df = df[df["emotion"]=="Negative emotion"]


In [ ]:
tweets = df['tweet'].to_string()
print(tweets)

In [ ]:
type(tweets)

In [ ]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
tokenized_tweets = tokenizer.tokenize(tweets)

tokenized_tweets = [x.lower() for x in tokenized_tweets]
sw = stopwords.words('english')
tokenized_tweets = [token for token in tokenized_tweets if token not in sw]

In [ ]:
tokenized_tweets

In [ ]:
freq = FreqDist(tokenized_tweets)
freq.most_common(50)

#### Stemming

In [ ]:
p_stemmer = nltk.stem.PorterStemmer()
s_stemmer = nltk.stem.SnowballStemmer(language="english")

In [ ]:
p_stemmer.stem(tweets)
s_stemmer.stem(tweets)

In [ ]:

for word in tweets:
    p_word = p_stemmer.stem(word)
    s_word = s_stemmer.stem(word)
    
    if p_word != s_word:
        print(word, p_word, s_word)

In [ ]:

tweets = [p_stemmer.stem(word) for word in tweets]
fdist = FreqDist(tweets)
plt.figure(figsize=(10, 10))
fdist.plot(30);


#### Lemmatizing

In [ ]:
def lemmatize_tokens(tokens):


    lemmatizer = WordNetLemmatizer()
    
    # Lemmatize each token and store the result in a new list
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return lemmatized_tokens

In [ ]:
# Lemmatizing positive tweet tokens
tweet_tokens_lemm = lemmatize_tokens(tokenized_tweets)

# Displaying most common 50 tokens
freq = FreqDist(tokenized_tweets)
freq.most_common(50)

### Feature Engineering

In [ ]:
# Feature engineering to have three categories: "Google," "Apple," and "Unknown"
df['brand'] = df['products'].replace({
    'iPhone': 'Apple',
    'iPad or iPhone App': 'Apple',
    'iPad': 'Apple',
    'Google': 'Google',
    'Unknown': 'Unknown',
    'Android': 'Google',
    'Apple': 'Apple',
    'Android App': 'Google',
    'Other Google product or service': 'Google',
    'Other Apple product or service': 'Apple'
})

# Drop the original "products" column
df.drop(columns=['products'], inplace=True)

# Display unique entries in the "brand" column
df['brand'].unique()

In [ ]:
# Filtering the DataFrame to select tweets associated with Apple
apple_tweets_df = df[df['brand'] == 'Apple']

# Converting the selected tweets to a list
apple_tweets_list = df['tweet'].tolist()

# Display the first few elements of the list
apple_tweets_list[:5]

In [ ]:

apple_tweets_string = ' '.join(apple_tweets_list)
print(apple_tweets_string)

In [ ]:
#tokenize tweets
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
apple_tweets_tokens = tokenizer.tokenize(apple_tweets_string)
apple_tweets_tokens = [token.lower() for token in sample_doc]
sw = stopwords.words('english')
apple_tweets_tokens = [token for token in sample_doc if token not in sw]

freq = FreqDist(apple_tweets_tokens)
freq.most_common(25)


In [ ]:
type(apple_tweets_tokens)


In [ ]:
wordcloud = WordCloud().generate(apple_tweets_tokens)


plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

### Modeling

#### Preprocessing

In [ ]:
# Getting the unique entities in "emotion" column
df['emotion'].unique()

In [ ]:
# Creating a binary DataFrame with no "Neutral emotion" category
binary_df =df[df['emotion'].isin(['Negative emotion', 'Positive emotion'])]

# Confirming the changes
binary_df['emotion'].unique()

In [ ]:

binary_emotion_mapping = {'Positive emotion': 1, 'Negative emotion': 0}

# Mapping the 'emotion' column to numerical values
binary_df['emotion'] = binary_df['emotion'].map(binary_emotion_mapping)

binary_df['emotion'].unique()

In [ ]:
# Checking if we have class imbalance
binary_df['emotion'].value_counts(normalize=True) * 100

#### Train Test Split

In [ ]:
X = binary_df['tweet']
y = binary_df['emotion']

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Creating a pipeline
clf_pipe = imblearn.pipeline.Pipeline([
    ('vectorizer', TfidfVectorizer(tokenizer=tokenize_text, 
                                   stop_words=None)), 
    ('os', RandomOverSampler(random_state=42)),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42, 
                                   criterion='entropy', max_depth=None, 
                                   min_samples_leaf=2))])
# Fitting the pipeline on the training data
clf_pipe.fit(X_train, y_train)

# Making predictions on the test data
y_pred = clf_pipe.predict(X_test)